this only works mediawiki wikis. I think almost every popular wiki uses this, and it's what you see in your head when you hear "wiki", ex: wikipedia uses mediawiki, fandom uses mediawiki, random wiki you can think of uses mediawiki, etc.

In [ ]:
import requests
import asyncio
import aiohttp
import aiofiles
from bs4 import BeautifulSoup, Comment
import os
import shutil
from slugify import slugify # python-slugify because apparently slugify is also a thing

In [142]:
baseurl = "https://criminalminds.fandom.com"  # i have to separate them for the href idk how else to do it
wikipath = "/wiki"
ignorecomments = True
discardtags = ["style", "script"]
folderpath = "./ignore"

In [143]:
allpages = baseurl + wikipath + "/Special:AllPages"
bads = ["AllPages", "SpecialPages", "Special:"]

pageset = []

currpage = requests.get(allpages)
currpage = BeautifulSoup(currpage.content, "html.parser")

pagenum = 1
nexturl = None  # housekeeping
while currpage is not None:
    print(f"page {pagenum}: {allpages if not nexturl else nexturl}")
    pagenum += 1
    for pageli in currpage.select_one(".mw-allpages-body").find_all("li"):
        liclass = pageli.get("class")
        if liclass is not None and liclass[0] == "allpagesredirect":
            continue
        href = pageli.select_one("a")["href"]

        if href and not any(bad in href for bad in bads):
            pageset.append(baseurl + href)

    nav = currpage.select_one(".mw-allpages-nav")
    currpage = None
    if nav is not None:
        for navoption in nav.find_all("a"):  # if no "a" should still only be a tags
            if "Next page" in navoption.string:
                nexturl = baseurl + navoption["href"]
                currpage = requests.get(nexturl)
                currpage = BeautifulSoup(currpage.content, "html.parser")
                break

print(len(pageset), len(set(pageset)))
if len(pageset) != len(set(pageset)):
    print("some duplicates made their way through. my code (or the wiki) has a bug")

page 1: https://criminalminds.fandom.com/wiki/Special:AllPages
page 2: https://criminalminds.fandom.com/wiki/Special:AllPages?from=Behind+the+Scenes%2FTake+4
page 3: https://criminalminds.fandom.com/wiki/Special:AllPages?from=Christie+Lynn+Smith
page 4: https://criminalminds.fandom.com/wiki/Special:AllPages?from=David+Wells
page 5: https://criminalminds.fandom.com/wiki/Special:AllPages?from=Fran%C3%A7ois+Chau
page 6: https://criminalminds.fandom.com/wiki/Special:AllPages?from=J.P.+Giuliotti
page 7: https://criminalminds.fandom.com/wiki/Special:AllPages?from=Johnny+Lewis
page 8: https://criminalminds.fandom.com/wiki/Special:AllPages?from=Lex+Medlin
page 9: https://criminalminds.fandom.com/wiki/Special:AllPages?from=Medalion+Rahimi
page 10: https://criminalminds.fandom.com/wiki/Special:AllPages?from=Paul+Carafotes
page 11: https://criminalminds.fandom.com/wiki/Special:AllPages?from=Robert+Ressler
page 12: https://criminalminds.fandom.com/wiki/Special:AllPages?from=Slade+Pearce
page 13: h

In [ ]:
def prepare_name(url):
    name = url.split(f"{wikipath}/")[1].split("?")[
        0
    ]  # hope ? isn't in the actual page name part
    name = slugify(name, lowercase=False)
    return name


def prune_html(rawhtml):
    soup = BeautifulSoup(rawhtml, "html.parser")
    if len(discardtags) > 0:  # because soup([]) gets everything
        for element in soup(discardtags):
            element.extract()
    if ignorecomments:
        comments = soup.find_all(string=lambda text: isinstance(text, Comment))
        for comment in comments:
            comment.extract()
    html = str(soup)
    return html


async def fetch_and_save(session, url):
    filename = prepare_name(url)
    async with session.get(url) as response, aiofiles.open(
        f"{folderpath}/{filename}.html", "w"
    ) as file:
        content = await response.text()
        content = prune_html(content)
        return await file.write(content)


async def fetch_and_save_all(links):
    async with aiohttp.ClientSession(requote_redirect_url=False) as session:
        # parameter is necessary cuz if u have & in link, fandom replaces it with %26 and redirects you, but this library turns it back into a &........
        tasks = [fetch_and_save(session, url) for url in links]
        return await asyncio.gather(*tasks)


def init_folder(reset=True):
    if reset and os.path.exists(folderpath):
        shutil.rmtree(folderpath)
    if not os.path.exists(folderpath):
        os.makedirs(folderpath)

In [ ]:
init_folder(reset=False)
try:
    asyncio.run(fetch_and_save_all(pageset))
except RuntimeError as e:
    if "running event loop" in str(e):
        await fetch_and_save_all(pageset)
print("done")

done


C:\Users\Andrew\AppData\Local\Temp\ipykernel_30480\1484359785.py:6: RuntimeWarning: coroutine 'fetch_and_save_all' was never awaited
  await fetch_and_save_all(pageset)
